#IMPORTING AND DEFINING PARAMETERS

In [1]:
import numpy as np
import tensorflow as tf

# This is enabled so that we can use pre-defined functions inside the tensorflow.keras.Model()
# In this case, we use the function "create_look_ahead_mask"
tf.config.run_functions_eagerly(True)

# The values below are suggested in the GPT-2 paper
embed_dim = 768
num_head = 12
seq_len = 5
key_dim = embed_dim//num_head

#TEXT PRE-PROCESSING, VOCABULARY CREATION & TOKENIZATION

In [10]:
with open('SherlockHolmesStory.txt', 'r', encoding='utf-8') as file:
  raw_text = file.read()
text = ''.join(raw_text.split('\n'))

altered_text = ''
for word in text.split():
  altered_word = ''
  for character in word:
    if character not in {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                         'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
                         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}:
      altered_word += ' ' + character + ' '
    else: altered_word += character
  altered_text += altered_word + ' ' if altered_word[-1] != ' ' else altered_word

vocab_size = 0
vocabulary = {'<PAD>': 0}
vocab_size += len(vocabulary)
for word in altered_text.split():
  if word not in vocabulary:
    vocabulary[word] = vocab_size
    vocab_size += 1
print(vocab_size, vocabulary)

tokenized_text = [vocabulary[i] for i in altered_text.split()]
print([i for i in altered_text.split()])
print(tokenized_text)

8428 {'<PAD>': 0, 'To': 1, 'Sherlock': 2, 'Holmes': 3, 'she': 4, 'is': 5, 'always': 6, 'the': 7, 'woman': 8, '.': 9, 'I': 10, 'have': 11, 'seldom': 12, 'heard': 13, 'him': 14, 'mention': 15, 'her': 16, 'under': 17, 'any': 18, 'other': 19, 'name': 20, 'In': 21, 'his': 22, 'eyes': 23, 'eclipses': 24, 'and': 25, 'predominates': 26, 'whole': 27, 'of': 28, 'sex': 29, 'It': 30, 'was': 31, 'not': 32, 'that': 33, 'he': 34, 'felt': 35, 'emotion': 36, 'akin': 37, 'to': 38, 'love': 39, 'for': 40, 'Irene': 41, 'Adler': 42, 'All': 43, 'emotions': 44, ',': 45, 'one': 46, 'particularly': 47, 'were': 48, 'abhorrent': 49, 'cold': 50, 'precise': 51, 'but': 52, 'admirably': 53, 'balanced': 54, 'mind': 55, 'He': 56, 'take': 57, 'it': 58, 'most': 59, 'perfect': 60, 'reasoning': 61, 'observing': 62, 'machine': 63, 'world': 64, 'has': 65, 'seen': 66, 'as': 67, 'a': 68, 'lover': 69, 'would': 70, 'placed': 71, 'himself': 72, 'in': 73, 'false': 74, 'position': 75, 'never': 76, 'spoke': 77, 'softer': 78, 'passio

#DATASET CREATION

In [3]:
broken_sentences = []
X, Y = [], []
for i in range(0, len(tokenized_text)):
  broken_sentences.append(tokenized_text[i:i+seq_len+1])

for item in broken_sentences:
  X.append(item[0:seq_len])
  Y.append([item[-1]])
X_Train = np.array(tf.keras.preprocessing.sequence.pad_sequences(X, maxlen = seq_len, padding = 'post'))
Y_Train = np.array(Y)
X_Train.shape, Y_Train.shape, X_Train, Y_Train

((128739, 5),
 (128739, 1),
 array([[   1,    2,    3,    4,    5],
        [   2,    3,    4,    5,    6],
        [   3,    4,    5,    6,    7],
        ...,
        [2209, 1252,    9,    0,    0],
        [1252,    9,    0,    0,    0],
        [   9,    0,    0,    0,    0]], dtype=int32),
 array([[6],
        [7],
        [8],
        ...,
        [9],
        [9],
        [9]]))

##DEFINING MODEL

In [ ]:
# Look ahead mask for restricting the architecture to attend to future tokens
def create_look_ahead_mask(seq_len):
  mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  return 1-mask
look_ahead_mask = create_look_ahead_mask(seq_len)

# Function to extract the last words' embedding from the matrix of word embeddings in previous layer
def expand_last_timestep(x):
  seq_len = tf.shape(x)[1]
  return tf.expand_dims(x[:, seq_len - 1, :], axis=1)

# Defining the 1-layer Transformer model as said in GPT-2 paper
def MODEL(seq_len, embed_dim, num_head, key_dim, vocab_size):
  # Input
  inp = tf.keras.Input((seq_len,))
  length = seq_len

  # Token/Word Embedding
  TokenEmbedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)(inp)

  # Position Embedding
  positions = tf.range(start=0, limit=length, delta=1)
  PositionEmbedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)(positions)

  # Merged Token & Position Embedding
  inputs = TokenEmbedding + PositionEmbedding

  ##############################################################################==LAYER 1

  # Key X Weight matrix
  key_1 = tf.keras.layers.Dense(embed_dim)(inputs)

  # Query X Weight matrix
  query_1 = tf.keras.layers.Dense(embed_dim)(inputs)

  # Value X Weight matrix
  value_1 = tf.keras.layers.Dense(embed_dim)(inputs)

  # Multi-head attention layer
  multi_head_attention_1 = tf.keras.layers.MultiHeadAttention(num_heads = num_head, key_dim = key_dim)
  multi_head_attention_1 = multi_head_attention_1(
    query=query_1,
    value=value_1,
    key=key_1,
    attention_mask=look_ahead_mask[tf.newaxis, :, :]
  )

  # Add + Layer Normalize
  add_11 = inputs + multi_head_attention_1
  layerNorm_11 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(add_11)

  # Feed forward neural network
  ffn_1 = tf.keras.Sequential(
      [
          tf.keras.layers.Dense(units=4*embed_dim, activation="relu"),
          tf.keras.layers.Dense(embed_dim),
      ]
      )(layerNorm_11)

  # Add + Layer Normalize
  add_12 = layerNorm_11 + ffn_1
  layerNorm_12 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(add_12)

  # Softmax over vocabulary
  out = tf.keras.layers.Dense(units=vocab_size, activation="softmax")(layerNorm_12)

  # Extract the last words' embedding from the matrix of word embeddings in previous layer
  out = tf.keras.layers.Lambda(expand_last_timestep, output_shape=(1, vocab_size))(out)

  # Defining model
  model = tf.keras.Model(inputs=inp, outputs=out)

  # Compiling the model using loss = "sparse_categorical_crossentropy", since, I am not encoding the output (I am using the tokens itself as y_actual)
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      optimizer="adam"
  )
  return model

pre_trained_model = MODEL(seq_len, embed_dim, num_head, key_dim, vocab_size)
pre_trained_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 5)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 5, 768)         │      6,472,704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 5, 768)         │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 5, 768)         │        590,592 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 5, 768)         │        590,592 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 5, 768)         │        590,592 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 5, 768)         │      2,362,368 │ dense[0][0],           │
│ (MultiHeadAttention)      │                        │                │ dense_1[0][0],         │
│                           │                        │                │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 5, 768)         │              0 │ add[0][0],             │
│                           │                        │                │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 5, 768)         │          1,536 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 5, 768)         │      4,722,432 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 5, 768)         │              0 │ layer_normalization[0… │
│                           │                        │                │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 5, 768)         │          1,536 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 5, 8428)        │      6,481,132 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 1, 8428)        │              0 │ dense_5[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,813,484 (83.21 MB)

 Trainable params: 21,813,484 (83.21 MB)

 Non-trainable params: 0 (0.00 B)

##MODEL TRAINING

In [ ]:
pre_trained_model.fit(X_Train, Y_Train, epochs=100, batch_size=256)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


503/503 ━━━━━━━━━━━━━━━━━━━━ 218s 424ms/step - loss: 5.6405
Epoch 2/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 215s 428ms/step - loss: 4.2795
Epoch 3/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 429ms/step - loss: 3.7702
Epoch 4/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 215s 427ms/step - loss: 3.4001
Epoch 5/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 427ms/step - loss: 3.0278
Epoch 6/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 261s 426ms/step - loss: 2.6506
Epoch 7/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 426ms/step - loss: 2.2850
Epoch 8/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 426ms/step - loss: 2.0088
Epoch 9/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 215s 427ms/step - loss: 1.7658
Epoch 10/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 428ms/step - loss: 1.5569
Epoch 11/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 428ms/step - loss: 1.3798
Epoch 12/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 263s 430ms/step - loss: 1.2393
Epoch 13/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 430ms/step - loss: 1.1162
Epoch 14/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 261s 428ms/step - loss: 1.0126
Epoch 15/20


##SAVE MODEL

In [ ]:
pre_trained_model.save('SHolmes_NWP_PRETRAINED.keras')

##LOAD SAVED MODEL

In [4]:
# To include lambda layer in the loaded model
tf.keras.config.enable_unsafe_deserialization()

# If there is a custom function layer defined in the actual model, then the
# definition of the function is to be redefined and the definition is to be
# mentioned in the custom_objects parameter of the keras.models.load_model()
def expand_last_timestep(x):
  seq_len = tf.shape(x)[1]
  return tf.expand_dims(x[:, seq_len - 1, :], axis=1)

uploaded_model = tf.keras.models.load_model("SHolmes_NWP_PRETRAINED.keras",
                                            custom_objects={'expand_last_timestep': expand_last_timestep})
uploaded_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 5, 768)    │  6,472,704 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 5, 768)    │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 5, 768)    │    590,592 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5, 768)    │    590,592 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 5, 768)    │    590,592 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 768)    │  2,362,368 │ dense[0][0],      │
│ (MultiHeadAttentio… │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 5, 768)    │          0 │ add[0][0],        │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 5, 768)    │      1,536 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 5, 768)    │  4,722,432 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 5, 768)    │          0 │ layer_normalizat… │
│                     │                   │            │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 768)    │      1,536 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 5, 8428)   │  6,481,132 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1, 8428)   │          0 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 65,440,454 (249.64 MB)

 Trainable params: 21,813,484 (83.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 43,626,970 (166.42 MB)

##TRAIN LOADED MODEL

In [5]:
uploaded_model.fit(X_Train, Y_Train, epochs=20, batch_size=256)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


503/503 ━━━━━━━━━━━━━━━━━━━━ 211s 410ms/step - loss: 0.1906
Epoch 2/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 258s 412ms/step - loss: 0.2015
Epoch 3/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 261s 411ms/step - loss: 0.2126
Epoch 4/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 206s 410ms/step - loss: 0.2108
Epoch 5/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 206s 410ms/step - loss: 0.2016
Epoch 6/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 411ms/step - loss: 0.1761
Epoch 7/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 411ms/step - loss: 0.1952
Epoch 8/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 206s 410ms/step - loss: 0.2141
Epoch 9/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 206s 410ms/step - loss: 0.1993
Epoch 10/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 206s 410ms/step - loss: 0.1790
Epoch 11/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 410ms/step - loss: 0.1725
Epoch 12/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 410ms/step - loss: 0.2193
Epoch 13/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 262s 410ms/step - loss: 0.2037
Epoch 14/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 261s 409ms/step - loss: 0.1755
Epoch 15/20


In [6]:
uploaded_model.save('SHolmes_NWP_PRETRAINED.keras')

#MODEL TESTING

In [5]:
# Testing the model on a already trained item
test_id = 210
rev_vocab = {val:key for key, val in vocabulary.items()}

predicted_tokens = np.squeeze(uploaded_model.predict(np.expand_dims(X_Train[test_id], axis=0)))
predicted_string = [rev_vocab[np.argmax(i)] for i in predicted_tokens]

print("Input text           :", ' '.join([rev_vocab[i] for i in X_Train[test_id]]))
print("Actual output text   :", ' '.join([rev_vocab[i] for i in Y_Train[test_id]]))
print("Predicted output text:", rev_vocab[np.argmax(predicted_tokens)])

/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step
Input text           : And yet there was but
Actual output text   : one
Predicted output text: one


##INFERENCE

In [14]:
text = 'I have seldom heard him mention'
n = 20

# Generate next words
for i in range(n):
  tokenized_text = [vocabulary[i] for i in text.split()]
  padded_text = np.array(tf.keras.preprocessing.sequence.pad_sequences([tokenized_text], maxlen = seq_len, padding = 'post'))
  prediction = np.squeeze(uploaded_model.predict(np.expand_dims(padded_text[0], axis=0)))

  string = [rev_vocab[np.argmax(i)] for i in prediction]
  print("Input text           :", ' '.join([rev_vocab[i] for i in tokenized_text]))
  print("Predicted output text:", rev_vocab[np.argmax(prediction)])
  text = text + ' ' + rev_vocab[np.argmax(prediction)]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Input text           : I have seldom heard him mention
Predicted output text: her
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Input text           : I have seldom heard him mention her
Predicted output text: under
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Input text           : I have seldom heard him mention her under
Predicted output text: any
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Input text           : I have seldom heard him mention her under any
Predicted output text: other
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Input text           : I have seldom heard him mention her under any other
Predicted output text: name
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Input text           : I have seldom heard him mention her under any other name
Predicted output text: .
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Input text           : I have seldom heard him mention her under any other name .
Predicted output text: In
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Input text         